In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
root='../../../../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
#cfg.data.train.palette = palette

cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'split_train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'split_val.json' # test json 정보
cfg.data.val.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash_wandb_add_val'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

NameError: name 'wandb' is not defined

In [11]:
cfg.runner.meta = dict(type = "exp_name",
                       exp_name= "exp_name")

In [5]:
import wandb
wandb.login()
wandb.init(project = "object-detection", name = 'exp')

cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={'project' : 'object-detection',
                      'name' : "workflow_add_val_dataset_add_val"},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=100)]


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [9]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [14]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 3906, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3179  | 1 [Paper]     | 5213  | 2 [Paper pack]  | 707   | 3 [Metal]   | 764   | 4 [Glass]    | 727   |
| 5 [Plastic]       | 2375  | 6 [Styrofoam] | 966   | 7 [Plastic bag] | 4114  | 8 [Battery] | 138   | 9 [Clothing] | 374   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [29]:
cfg.data.train

{'type': 'CocoDataset',
 'ann_file': '../../../../../dataset/split_train.json',
 'img_prefix': '../../../../../dataset/',
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'LoadAnnotations', 'with_bbox': True},
  {'type': 'Resize', 'img_scale': (512, 512), 'keep_ratio': True},
  {'type': 'RandomFlip', 'flip_ratio': 0.5},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_rgb': True},
  {'type': 'Pad', 'size_divisor': 32},
  {'type': 'DefaultFormatBundle'},
  {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}],
 'classes': ('General trash',
  'Paper',
  'Paper pack',
  'Metal',
  'Glass',
  'Plastic',
  'Styrofoam',
  'Plastic bag',
  'Battery',
  'Clothing')}

In [10]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-10-16 14:18:51,298 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2024-10-16 14:18:51,300 - mmcv - INFO - load model from: torchvision://resnet50
2024-10-16 14:18:51,301 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2024-10-16 14:18:51,452 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2024-10-16 14:18:51,470 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-10-16 14:18:51,500 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2024-10-16 14:18:51,507 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [11]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2024-10-16 14:18:53,134 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-10-16 14:18:53,159 - mmdet - INFO - Start running, host: root@instance-12243, work_dir: /data/ephemeral/home/KSG-git/level2-objectdetection-cv-20/code/baseline/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash_wandb_add_val
2024-10-16 14:18:53,160 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MM

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2024-10-16 14:18:56,061 - mmdet - WARNING - No meta information found in the runner. 


Exception: input type is not supported.

wandb: WARNING Fatal error while uploading data. Some run data will not be synced, but it will still be written to disk. Use `wandb sync` at the end of the run to try uploading.
